In [1]:
import numpy as np
import pandas as pd
import re

# This is for all bavaria, the other notebook is for munich only.

# --- 1) Read the Data ---
census_2024_df = pd.read_csv("census_data.csv", sep=",", dtype={"commune_id": str})  # keep strings as strings
n_distinct = census_2024_df.iloc[:, 0].nunique(dropna=True)
print("Number of distinct comune ids:", n_distinct)
print("In total, the population for Oberbayern, Niederbayern and Schwaben is:", census_2024_df['weight'].sum())
# mask_munich = census_df["commune_id"].astype(str).str.startswith("9162")
# total_weight = census_df.loc[mask_munich, "weight"].sum(min_count=1)  # returns NaN if none match
# print("There are ", total_weight, " people in Munich")

# sex_weights_df = census_df[mask_munich]
# age_classes_target_classes = sorted(census_df.loc[mask_munich, "age_class"].unique())
population_2045 = pd.read_csv("population_oberbayern_niederbayern_schwaben_2043.csv", sep=",", dtype={"kommunen_id_start": str})

Number of distinct comune ids: 1098
In total, the population for Oberbayern, Niederbayern and Schwaben is: 8021702


In [3]:
# Ensure types are as expected
census_2024_df["commune_id"] = census_2024_df["commune_id"].astype(str).str.strip()
census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce")

# Drop rows without an ID
census_2024_df = census_2024_df[census_2024_df["commune_id"].notna() & (census_2024_df["commune_id"] != "")]

# Build: commune_id -> total population (sum of weights)
census_2_pop = (
    census_2024_df.groupby("commune_id", dropna=False)["weight"]
             .sum(min_count=1)         # keeps NaN if all weights were NaN for an ID
             .fillna(0)                # optional: treat all-NaN as 0
             .to_dict()
)

In [8]:
# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
# 1) population_2045: one row per 5-digit prefix
#    Must have columns: kommunen_id_start (str), Name_norm (str), population_2043_abs (int)
# population_2045 = pd.read_csv(
#     "kommunen_oberbayern_niederbayern_schwaben_2043_with_names_start_only.csv",
#     dtype={"kommunen_id_start": str}
population_2045 = population_2045[["kommunen_id_start","Name_norm","population_2043_abs"]].copy()

# 2) census input: either a dict census_2_pop or a census_df to build it
# If you already have census_2_pop, comment out the block that builds it.

try:
    census_2_pop  # dict exists
except NameError:
    # Build from census_df
    census_2024_df = pd.read_csv("census_data_2024.csv", dtype={"commune_id": str})
    census_2024_df["commune_id_norm"] = normalize_id(census_2024_df["commune_id"])
    census_2024_df = census_2024_df[census_2024_df["commune_id_norm"].notna()].copy()
    census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce").fillna(0)

    # Sum weights per commune (current population by commune)
    census_2_pop = (census_2024_df.groupby("commune_id_norm")["weight"]
                    .sum(min_count=1).fillna(0).to_dict())

# -------------------- Build commune-level frame --------------------
# Turn census_2_pop dict into a DataFrame
commune_now = (pd.Series(census_2_pop, name="population_2024_kommune")
               .rename_axis("kommunen_id")  # 12-digit id
               .reset_index())

# Derive 5-digit prefix for each commune
commune_now["kommunen_id"] = normalize_id(commune_now["kommunen_id"])
commune_now = commune_now[commune_now["kommunen_id"].notna()].copy()
commune_now["kommunen_id_start"] = commune_now["kommunen_id"].str[:5]

# -------------------- Aggregate current pop by prefix --------------------
grp_now = (commune_now.groupby("kommunen_id_start", dropna=True)["population_2024_kommune"]
           .sum(min_count=1).rename("population_2024_gemeinde")
           .reset_index())

# Attach the group total and the PDF 2043 totals & name (by prefix)
work = (commune_now
        .merge(grp_now, on="kommunen_id_start", how="left")
        .merge(population_2045.rename(columns={"population_2043_abs":"population_2043_gemeinde",
                                               "Name_norm":"name"}),
               on="kommunen_id_start", how="left"))

# -------------------- Compute shares & scaled 2043 --------------------
# Guard against division by zero: if the group total is 0 or NaN, set share=0 and scaled=0
den = work["population_2024_gemeinde"].replace({0: np.nan})
share = work["population_2024_kommune"] / den
work["weight_of_single_kommune_among_gemeinde"] = (share.fillna(0) * 100)  # percent
work["population_2043_gemeinde"] = pd.to_numeric(work["population_2043_gemeinde"], errors="coerce").fillna(0)

work["scaled_population_2043_kommune"] = (
    work["population_2043_gemeinde"] * (work["weight_of_single_kommune_among_gemeinde"] / 100.0)
).round(0).astype(int)

# -------------------- Final columns & save --------------------
result = work[[
    "kommunen_id",                        # 12-digit
    "kommunen_id_start",                  # 5-digit prefix
    "name",                               # from the prefix row
    "population_2024_kommune",            # current pop by commune (from census_2_pop)
    "population_2024_gemeinde",           # sum over the prefix
    "weight_of_single_kommune_among_gemeinde",  # percent
    "population_2043_gemeinde",           # PDF 2043 per prefix
    "scaled_population_2043_kommune"      # allocated 2043 per commune
]].copy()

# Optional: tidy types / rounding
result["weight_of_single_kommune_among_gemeinde"] = result["weight_of_single_kommune_among_gemeinde"].round(6)

In [9]:
result_cleaned = result[["kommunen_id", "population_2024_kommune", "scaled_population_2043_kommune"]]
result_cleaned = result_cleaned.rename(
    columns={
        "population_2024_kommune": "pop_2024",
        "scaled_population_2043_kommune": "pop_2043",
    }
)

In [10]:
import pandas as pd
import numpy as np
import re

# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
# result_cleaned must have: kommunen_id, pop_2024, pop_2043
# census_df must have: commune_id, sex, age_class, weight
# If you already have them in memory, you can skip these reads.
# result_cleaned = pd.read_csv("result_cleaned.csv", dtype={"kommunen_id": str})
# census_df = pd.read_csv("census_data.csv", dtype={"commune_id": str})

# --- Normalize IDs and clean types
result_cleaned["kommunen_id"] = normalize_id(result_cleaned["kommunen_id"])
result_cleaned = result_cleaned[result_cleaned["kommunen_id"].notna()].copy()

census_2024_df["commune_id_norm"] = normalize_id(census_2024_df["commune_id"])
census_2024_df = census_2024_df[census_2024_df["commune_id_norm"].notna()].copy()
census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce").fillna(0)

# -------------------- 1) Enumerate the 26 combinations --------------------
all_combos = (
    census_2024_df[["sex", "age_class"]]
      .drop_duplicates()
      .sort_values(["sex", "age_class"])
      .reset_index(drop=True)
)
# Sanity: we expect 26 distinct combos total
n_combos = len(all_combos)
if n_combos != 26:
    print(f"WARNING: expected 26 (sex, age_class) combos, found {n_combos}. Proceeding with found combos.")

# -------------------- 2) Compute per-(commune_id, combo) weights & fractions --------------------
combo_weights = (
    census_2024_df
      .groupby(["commune_id_norm", "sex", "age_class"], dropna=False)["weight"]
      .sum(min_count=1)
      .reset_index()
)

totals_per_commune = (
    combo_weights
      .groupby("commune_id_norm", dropna=False)["weight"]
      .sum(min_count=1)
      .rename("total_weight_commune")
      .reset_index()
)

combo_weights = combo_weights.merge(totals_per_commune, on="commune_id_norm", how="left")

# Fraction per combo (guard against division by zero)
combo_weights["fraction_2024"] = (
    combo_weights["weight"] / combo_weights["total_weight_commune"].replace({0: np.nan})
).fillna(0.0)

# -------------------- 3) Ensure every commune has all combos (fill missing with 0) --------------------
communes = result_cleaned[["kommunen_id"]].drop_duplicates().rename(columns={"kommunen_id": "commune_id_norm"})
grid = communes.assign(key=1).merge(all_combos.assign(key=1), on="key").drop(columns="key")

grid = grid.merge(
    combo_weights[["commune_id_norm", "sex", "age_class", "fraction_2024"]],
    on=["commune_id_norm", "sex", "age_class"],
    how="left"
)
grid["fraction_2024"] = grid["fraction_2024"].fillna(0.0)

# OPTIONAL EVEN-SPLIT FALLBACK:
# If a commune has total_weight_commune == 0 (i.e., no data at all), you might want to split evenly across combos:
# zero_data_communes = grid.groupby("commune_id_norm")["fraction_2024"].sum().reset_index()
# zero_data_communes = zero_data_communes[zero_data_communes["fraction_2024"] == 0]["commune_id_norm"]
# grid.loc[grid["commune_id_norm"].isin(zero_data_communes), "fraction_2024"] = 1.0 / n_combos

# -------------------- 4) Expand result_cleaned (26 rows per commune) and scale pops --------------------
expanded = (
    grid.merge(
        result_cleaned.rename(columns={"kommunen_id": "commune_id_norm"}),
        on="commune_id_norm",
        how="left"
    )
)

expanded["pop_2024_by_combo"] = expanded["pop_2024"] * expanded["fraction_2024"]
expanded["pop_2043_by_combo"] = expanded["pop_2043"] * expanded["fraction_2024"]

# -------------------- 5) Validations --------------------
# A) Fractions per commune should sum to 1 (or 0 if truly no data and no fallback)
frac_sums = expanded.groupby("commune_id_norm")["fraction_2024"].sum().reset_index()
bad_fracs = frac_sums[~np.isclose(frac_sums["fraction_2024"], 1.0, atol=1e-8) & (frac_sums["fraction_2024"] != 0)]
if not bad_fracs.empty:
    print("WARNING: Some communes have fractions not summing to 1 (and not 0). Examples:")
    print(bad_fracs.head())

# B) Check that sums of distributed pops match originals (within tolerance)
check_2024 = (
    expanded.groupby("commune_id_norm")["pop_2024_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2024"]],
           on="commune_id_norm", how="left")
)
check_2024["diff_2024"] = check_2024["pop_2024_by_combo"] - check_2024["pop_2024"]
if not np.allclose(check_2024["diff_2024"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2024_by_combo does not perfectly sum back to pop_2024 for some communes.")

check_2043 = (
    expanded.groupby("commune_id_norm")["pop_2043_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2043"]],
           on="commune_id_norm", how="left")
)
check_2043["diff_2043"] = check_2043["pop_2043_by_combo"] - check_2043["pop_2043"]
if not np.allclose(check_2043["diff_2043"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2043_by_combo does not perfectly sum back to pop_2043 for some communes.")

# -------------------- 6) Final tidy --------------------
result_expanded = expanded[[
    "commune_id_norm",  # == kommunen_id (12 digits)
    "sex",
    "age_class",
    "fraction_2024",
    "pop_2024_by_combo",
    "pop_2043_by_combo",
    "pop_2024",
    "pop_2043",
]].rename(columns={"commune_id_norm": "kommunen_id"})

# (Optional) Round to integers if you need person counts:
# result_expanded["pop_2024_by_combo"] = result_expanded["pop_2024_by_combo"].round(0).astype(int)
# result_expanded["pop_2043_by_combo"] = result_expanded["pop_2043_by_combo"].round(0).astype(int)

# Save if desired
# result_expanded.to_csv("result_expanded_26_combos.csv", index=False)

In [11]:
result_expanded.head(20)

,kommunen_id,sex,age_class,fraction_2024,pop_2024_by_combo,pop_2043_by_combo,pop_2024,pop_2043
0,091610000000,female,0,0.015983,2254.0,2405.370527,141029,150500
1,091610000000,female,3,0.015642,2206.0,2354.147019,141029,150500
2,091610000000,female,6,0.019414,2738.0,2921.874224,141029,150500
3,091610000000,female,10,0.021414,3020.0,3222.812329,141029,150500
4,091610000000,female,15,0.012607,1778.0,1897.404080,141029,150500
5,091610000000,female,18,0.009381,1323.0,1411.847918,141029,150500
6,091610000000,female,20,0.026051,3674.0,3920.732615,141029,150500
7,091610000000,female,25,0.033759,4761.0,5080.731623,141029,150500
8,091610000000,female,30,0.074077,10447.0,11148.582916,141029,150500
9,091610000000,female,40,0.064561,9105.0,9716.459026,141029,150500


In [12]:
final_result = result_expanded.rename(
    columns={
        "kommunen_id": "commune_id",
        "pop_2043_by_combo": "weight",
    }
)

In [13]:
final_result=final_result[["commune_id", "sex","age_class","weight"]]

In [ ]:
processed_pop_munich_2045 = pd.read_csv("processed_pop_munich_2045.csv")

In [18]:
processed_pop_munich_2045.head()

,commune_id,age_start,age_end,total_2045,predicted_female_share_2045,female_2045,male_2045
0,91620000000,0,2,56184,0.487231,27375,28809
1,91620000000,3,5,50839,0.494434,25137,25702
2,91620000000,6,9,62569,0.490813,30710,31859
3,91620000000,10,14,74144,0.488693,36234,37910
4,91620000000,15,17,44982,0.483929,21768,23214


In [ ]:
import pandas as pd
import numpy as np
import re

# ---------- Helpers ----------
def normalize_id(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.strip().str.replace(r"\D", "", regex=True).str.zfill(12))

def normalize_sex(s: pd.Series) -> pd.Series:
    # map a variety of spellings to {'female','male'}; everything else -> lowercased as-is
    mapping = {
        "f":"female","w":"female","weiblich":"female","female":"female",
        "m":"male","mann":"male","maennlich":"male","männlich":"male","male":"male"
    }
    return s.astype(str).str.strip().str.lower().map(mapping).fillna(s.astype(str).str.strip().str.lower())

# ---------- 0) Make safe copies ----------
result_final = final_result.copy()
processed_pop_munich_2045 = processed_pop_munich_2045.copy()

# ---------- 1) Normalize keys / types ----------
# result_final
if "kommunen_id" in result_final.columns and "commune_id" not in result_final.columns:
    result_final = result_final.rename(columns={"kommunen_id":"commune_id"})
result_final["commune_id"] = normalize_id(result_final["commune_id"])
result_final["sex"] = normalize_sex(result_final["sex"])
result_final["age_class"] = pd.to_numeric(result_final["age_class"], errors="coerce").astype("Int64")
result_final["weight"] = pd.to_numeric(result_final["weight"], errors="coerce")

# processed_pop_munich_2045
processed_pop_munich_2045["commune_id"] = normalize_id(processed_pop_munich_2045["commune_id"])
processed_pop_munich_2045["age_start"] = pd.to_numeric(processed_pop_munich_2045["age_start"], errors="coerce").astype("Int64")

# ---------- 2) Reshape Munich table to long by sex ----------
updates_long = (
    processed_pop_munich_2045
      .melt(id_vars=["commune_id","age_start"],
            value_vars=["female_2045","male_2045"],
            var_name="sex_var", value_name="weight_new")
      .assign(sex=lambda df: df["sex_var"].str.replace("_2045","",regex=False))
      .rename(columns={"age_start":"age_class"})
      [["commune_id","sex","age_class","weight_new"]]
)

# If there are accidental duplicates per key, collapse them (sum)
updates_long = (updates_long
                .groupby(["commune_id","sex","age_class"], as_index=False)["weight_new"]
                .sum())

# ---------- 3) Check key alignment BEFORE merge ----------
print("Distinct sex in result_final:", sorted(result_final["sex"].dropna().unique())[:10])
print("Distinct sex in updates_long:", sorted(updates_long["sex"].dropna().unique())[:10])

rf_keys = set(zip(result_final["commune_id"], result_final["sex"], result_final["age_class"]))
up_keys = set(zip(updates_long["commune_id"], updates_long["sex"], updates_long["age_class"]))
matches = rf_keys & up_keys
print("Potential key matches:", len(matches), "out of", len(up_keys), "update keys and", len(rf_keys), "result keys")

if len(matches) == 0:
    # Show a few example keys that fail to match
    only_in_updates = list(up_keys - rf_keys)[:10]
    only_in_result  = list(rf_keys - up_keys)[:10]
    print("Examples of update keys not in result_final:", only_in_updates)
    print("Examples of result_final keys not in updates:", only_in_result)

# ---------- 4) Merge & overwrite weight where we have a match ----------
merged = result_final.merge(
    updates_long,
    on=["commune_id","sex","age_class"],
    how="left"
)

# Keep old for reference if you want:
# merged["weight_old"] = merged["weight"]

updated_count = merged["weight_new"].notna().sum()
print("Rows that will be updated:", updated_count)

merged["weight"] = np.where(merged["weight_new"].notna(), merged["weight_new"], merged["weight"])
merged = merged.drop(columns=["weight_new"])

# This is your updated result_final:
result_final = merged

# --- Optional: verify Munich totals match the source after update
muc_id = "091620000000"
check = (
    result_final[result_final["commune_id"] == muc_id]
      .groupby("sex")["weight"].sum()
      .rename("sum_in_result_final")
      .to_frame()
      .join(
          updates_long[updates_long["commune_id"] == muc_id]
            .groupby("sex")["weight_new"].sum()
            .rename("sum_from_updates"),
          how="left"
      )
)
print(check)

Distinct sex in result_final: ['female', 'male']
Distinct sex in updates_long: ['female', 'male']
Potential key matches: 26 out of 26 update keys and 28548 result keys
Rows that will be updated: 26
        sum_in_result_final  sum_from_updates
sex                                          
female             937453.0            937453
male               892068.0            892068


In [35]:
result_final["weight"] = result_final["weight"].astype(int)

In [28]:
final_result_2 = result_final.copy()
final_result_2["commune_id"] = final_result_2["commune_id"].astype(str)
subset = final_result_2[final_result_2["commune_id"] == "091620000000"]

In [29]:
subset

,commune_id,sex,age_class,weight
26,091620000000,female,0,27375.0
27,091620000000,female,3,25137.0
28,091620000000,female,6,30710.0
29,091620000000,female,10,36234.0
30,091620000000,female,15,21768.0
31,091620000000,female,18,18692.0
32,091620000000,female,20,57209.0
33,091620000000,female,25,85019.0
34,091620000000,female,30,168660.0
35,091620000000,female,40,117174.0


In [44]:
result_final["commune_id"] = result_final["commune_id"].astype("string")

result_final.to_csv("census_data_2043.csv", index=False)

In [41]:
# 1) Column dtypes (pandas dtypes)
print(result_final.dtypes)

# 2) Per-cell Python type (by row) — creates a parallel DataFrame of type names
types_df = result_final.applymap(lambda x: type(x).__name__)
print(types_df.head())          # first rows of types
print(types_df.iloc[0])         # types of row 0


commune_id    object
sex           object
age_class      Int64
weight         int64
dtype: object
  commune_id  sex age_class weight
0        str  str       int    int
1        str  str       int    int
2        str  str       int    int
3        str  str       int    int
4        str  str       int    int
commune_id    str
sex           str
age_class     int
weight        int
Name: 0, dtype: object


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_12620/1332315755.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  types_df = result_final.applymap(lambda x: type(x).__name__)


In [40]:
print(census_2024_df.dtypes)

# 2) Per-cell Python type (by row) — creates a parallel DataFrame of type names
types_df = census_2024_df.applymap(lambda x: type(x).__name__)
print(types_df.head())          # first rows of types
print(types_df.iloc[0]) 

commune_id         object
sex                object
age_class           int64
weight              int64
commune_id_norm    object
dtype: object
  commune_id  sex age_class weight commune_id_norm
0        str  str       int    int             str
1        str  str       int    int             str
2        str  str       int    int             str
3        str  str       int    int             str
4        str  str       int    int             str
commune_id         str
sex                str
age_class          int
weight             int
commune_id_norm    str
Name: 0, dtype: object


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_12620/3392168973.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  types_df = census_2024_df.applymap(lambda x: type(x).__name__)
